# Functores Aplicativos

Tenemos el siguiente problema. No podemos expresar operaciones entre 2 instancias de un functor:

```haskell
aplicaEntreFunctores (+)  (Just 5)     (Just 11)
aplicaEntreFunctores (++) [1,2,3]      [4,5,6]
aplicaEntreFunctores (&&) (Right True) (Right False)
```

Lo que sí podemos hacer, es mapear un functor a un functor que contiene una función:

In [1]:
:t fmap (+) (Just 5) -- Just ((+) 5) = Just (5 +)

fmap (+) (Just 5) :: forall a. Num a => Maybe (a -> a)

Lo anterior, lo podríamos lograr si tuvieramos alguna de las siguientes 2 opciones:

- `apply :: f (a -> b) -> f a -> f b`
- `lift  :: (a -> b -> c) -> f a -> f b -> f c`

Además, necesitaríamos lo siguiente para crear functores:

- `pure :: a -> f a`

En esencia, lo que queremos lograr es esto:

![Ilustración de Aplicativos](applicative.png)

**¿Por qué no modificamos los functores para admitir lo anterior?**: No todos los functores pueden definir una función `pure`.

Puesto que los functores no van a cumplir nuestras demandas anteriores, creamos una nueva clase llamada `Aplicativo`. Sus funciones serán `pure`, `<*>` (que corresponde a `apply`) y `liftA2` que corresponde a `lift`.

_Nota: `<*>` debe definirse en función de `liftA2` y viceversa._

_Nota 2: `<*>` es infijo asociativo a la izquierda, con jerarquía 4._

In [2]:
-- fmap  :: (a -> b) -> f a -> f b
-- (<$>) :: (a -> b) -> f a -> f b

class Functor f => Aplicativo f where
    pure :: a -> f a
    
    infixl 4 <*>
    (<*>) :: f (a -> b) -> f a -> f b
    (<*>) = liftA2 id
    
    -- x -> x
    -- (a -> b -> c) -> f a -> f b -> f c
    -- (x -> x) = (a -> b -> c)
    -- (x -> x) = (a -> (b -> c))
    -- a = (b -> c) 
    -- (x -> x) = ((b -> c) -> (b -> c))
    -- x = (b -> c)
    -- ((b -> c) -> (b -> c)) = ((b -> c) -> (b -> c))
    
    -- f (a -> b) -> f a -> f b
    -- f (b -> c) -> f b -> f c
    
    -- (a1 -> b1 -> c1) = ((b1 -> c1) -> (b1 -> c1))
    -- ((b1 -> c1) -> (b1 -> c1)) = x -> x
    -- x = b1 -> c1
    -- y = b1 -> c1
    
    -- f a1 -> f b1 -> f c1
    -- f (a2 -> b2) -> f a2 -> f b2
    -- f (a2 -> b2) -> f a2 -> f b2
    -- a1 = a2 -> b2
    -- b1 = a2
    -- c1 = b2
    
    liftA2 :: (a -> b -> c) -> f a -> f b -> f c
    liftA2 f fa fb = (f <$> fa) <*> fb
    -- f (a -> b -> c)
    -- f (a -> (b -> c))
    -- f (b -> c)
    
    -- pure f :: f (a -> b -> c)
    --        :: f (b -> c)
    -- (a -> (b -> c)) -> f a -> f (b -> c)

In [3]:
:i Aplicativo

type Aplicativo :: (* -> *) -> Constraint
class Functor f => Aplicativo f where
  pure :: a -> f a
  (<*>) :: f (a -> b) -> f a -> f b
  liftA2 :: (a -> b -> c) -> f a -> f b -> f c
  {-# MINIMAL pure #-}
  	-- Defined at <interactive>:1:1

24/7.
Supports Textbook solut## Leyes

- Identidad: `pure id <*> x = x`.
- Composición: `pure (.) <*> x <*> y <*> z = x <*> (y <*> z)`.
- Homomorfismo: `pure f <*> pure x = pure (f x)`.
- Intercambio: `u <*> pure y = pure ($ y) <*> u`.

## Instancias

Definamos a las listas como instancias de aplicativos.

In [4]:
-- a -> [a]

-- [(a -> b)] -> [a] -> [b]
-- (a -> b -> c) -> [a] -> [b] -> [c]

instance Aplicativo [] where
    pure x = [x]
    fs <*> xs = [f x | f <- fs, x <- xs]
    liftA2 f xs ys = [f x y | x <- xs, y <- ys]


In [5]:
liftA2 (+) [1,2,3] [3,2,1]

[4,3,2,5,4,3,6,5,4]

Definamos a `Maybe` como una instancia de aplicativo.

In [6]:
-- a -> Maybe a

-- Maybe (a -> b) -> Maybe a -> Maybe b
-- (a -> b -> c) -> Maybe a -> Maybe b -> Maybe c

-- data Maybe a = Nothing | Just a

instance Aplicativo Maybe where
    pure x = Just x
    liftA2 f (Just x) (Just y) = Just (f x y)
    liftA2 _ _        _        = Nothing

Definamos a las funciones como instancias de aplicativos.

In [7]:
instance Aplicativo ((->) r) where
    pure a _ = a
    frab <*> fra = \r -> frab r (fra r)

En los Aplicativos, las instancias para los tipos dejan de ser necesariamente únicas. Veamos el ejemplo con listas:

In [8]:
(++) <$> ["Hola", "Lambda"] <*> ["Mundo", "Club"]
liftA2 (++) ["Hola", "Lambda"] ["Mundo", "Club"]

["HolaMundo","HolaClub","LambdaMundo","LambdaClub"]

["HolaMundo","HolaClub","LambdaMundo","LambdaClub"]

En la definición por defecto, nos generar todas las combinaciones de 2 listas bajo una función. Pero también sería un aplicativo válido el que resulta en lo siguiente:

```haskell
> (++) <$> ["Hola", "Lambda"] <*> ["Mundo", "Club"]
> ["HolaMundo", "LambdaClub"]
```

Define la segunda instancia de aplicativos para listas con el comportamiento anterior. Utiliza la función `zipWith`.

In [9]:
:t zipWith

zipWith :: forall a b c. (a -> b -> c) -> [a] -> [b] -> [c]

In [10]:
-- No hubo tiempo de verlo en la sesión, pero aquí está la solución:

newtype ZipList a = ZipList [a]

-- Primero debemos hacer que esta sea un Functor,
-- para poder ser un Aplicativo.
instance Functor ZipList where
    fmap f (ZipList xs) = ZipList (fmap f xs)
    
instance Aplicativo ZipList where
    pure x = ZipList [x]
    liftA2 f (ZipList xs) (ZipList ys) = ZipList (zipWith f xs ys)

In [17]:
(++) <$> ["Hola", "Lambda"] <*> ["Mundo", "Club"]

["HolaMundo","HolaClub","LambdaMundo","LambdaClub"]

## Casos de uso

- Punto medio entre functores y monoides.
- Parsear.

In [11]:
import Data.Char (isDigit, isLetter, isSpace)
import Text.Read (readMaybe)

data Usuario = Usuario { idUsuario :: Int
                       , edad      :: Int
                       , nombre    :: String
                       } deriving Show

parseaID :: String -> Maybe Int
parseaID = readMaybe . takeWhile isDigit

parseaEdad :: String -> Maybe Int
parseaEdad text = let (_:xs) = dropWhile (not . isSpace) text
                    in readMaybe $ takeWhile isDigit xs

parseaNombre :: String -> Maybe String
parseaNombre = Just . takeWhile isLetter . dropWhile (not . isLetter)

In [12]:
parseaID     "9 20 jpyamamoto"
parseaEdad   "9 20 jpyamamoto"
parseaNombre "9 20 jpyamamoto"

Just 9

Just 20

Just "jpyamamoto"

In [13]:
parseaUsuario :: String -> Maybe Usuario
parseaUsuario u = Usuario <$> parseaID u <*> parseaEdad u <*> parseaNombre u

In [14]:
parseaUsuario "1 20 jpyamamoto"
parseaUsuario "2 92 alonzochurch"
parseaUsuario "3 alanturing 41"

fmap parseaUsuario ["1 20 jpyamamoto", "2 92 alonzochurch", "3 alanturing 41"]

Just (Usuario {idUsuario = 1, edad = 20, nombre = "jpyamamoto"})

Just (Usuario {idUsuario = 2, edad = 92, nombre = "alonzochurch"})

Nothing

[Just (Usuario {idUsuario = 1, edad = 20, nombre = "jpyamamoto"}),Just (Usuario {idUsuario = 2, edad = 92, nombre = "alonzochurch"}),Nothing]

Concurrencia:

```haskell
#!/usr/bin/env stack
-- stack script --resolver lts-12.21
import Conduit
import UnliftIO

main :: IO ()
main = do
  write2Files
  runConduitRes $
    (sourceFile "file1.txt" *> sourceFile "file2.txt") .|
    sink

write2Files = runConcurrently $
      Concurrently (writeFile "file1.txt" "this is file 1")
   *> Concurrently (writeFile "file2.txt" "this is file 2")

sink = getZipSink $
      ZipSink (sinkFile "output1.txt")
   *> ZipSink (sinkFile "output2.txt")
```

In [15]:
:t (*>)

(*>) :: forall (f :: * -> *) a b. Applicative f => f a -> f b -> f b